<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 검증합니다.
- 특정 모델을 GridSearchCV를 사용해 성능을 최적화합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

In [2]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/boston.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [3]:
# 데이터 살펴보기
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
# 기초통계정보 확인
data.describe()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [5]:
# target 확인
target = 'medv'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]


**2) 정규화**

In [6]:
# 정규화
x = (x - x.min()) / (x.max() - x.min())
x.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338


**3) 학습용, 평가용 데이터 분리**

In [7]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2022, test_size=0.3)

# 4.모델 검증

- 여러 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 검증합니다.
- 하이퍼파라미터를 설정하지 않고 진행합니다.
- 각 모델의 성능 정보를 수집해 마지막에 비교합니다.

## 4.1.Linear Regression

- Linear Regression 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [9]:
# 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [10]:
# 선언하기
model = LinearRegression()

In [29]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)
cv_score

array([0.36715057, 0.67693985, 0.66740487, 0.73962399, 0.58467739,
       0.47964806, 0.68490351, 0.55404666, 0.77567643, 0.83437924])

In [30]:
# 결과수집
result = {}
result['Linear Regression'] = cv_score.mean()
result

{'Linear Regression': 0.6364450566848314}

## 4.2.KNN

- KNN 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [13]:
# 불러오기
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

In [14]:
# 선언하기
model = KNeighborsRegressor()

In [31]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)
cv_score

array([0.36715057, 0.67693985, 0.66740487, 0.73962399, 0.58467739,
       0.47964806, 0.68490351, 0.55404666, 0.77567643, 0.83437924])

In [32]:
# 결과수집
result['KNN'] = cv_score.mean()
result

{'Linear Regression': 0.6364450566848314, 'KNN': 0.6364450566848314}

## 4.3.Decision Tree

- Decision Tree 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022을 지정합니다.

In [33]:
# 불러오기
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [34]:
# 선언하기
model = DecisionTreeRegressor(random_state=2020)

In [35]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [36]:
# 결과수집
result['Decision Tree'] = cv_score.mean()
result

{'Linear Regression': 0.6364450566848314,
 'KNN': 0.6364450566848314,
 'Decision Tree': 0.6846896542421028}

## 4.4.SVM

- SVM 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [38]:
# 불러오기
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 

In [39]:
# 선언하기
model = SVR()

In [49]:
# 성능검증
kf = KFold(n_splits=20)
cv_score = cross_val_score(model, x_train, y_train, cv=kf)

In [50]:
# 결과수집
result['SVM'] = cv_score.mean()
result

{'Linear Regression': 0.6364450566848314,
 'KNN': 0.6364450566848314,
 'Decision Tree': 0.6846896542421028,
 'SVM': 0.5470482549697075}

## 4.5.Random Forest

- Random Forest 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022을 지정합니다.

In [51]:
# 불러오기
from sklearn.ensemble import RandomForestRegressor

In [52]:
# 선언하기
model = RandomForestRegressor(random_state=2022)

In [53]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [54]:
# 결과수집
result['Random Forest'] = cv_score.mean()
result

{'Linear Regression': 0.6364450566848314,
 'KNN': 0.6364450566848314,
 'Decision Tree': 0.6846896542421028,
 'SVM': 0.5470482549697075,
 'Random Forest': 0.8589945611695077}

## 4.6.XGBoost

- XGBoost 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022를 지정합니다.

In [61]:
# 불러오기
from xgboost import XGBRegressor

In [62]:
# 선언하기
model = XGBRegressor(eval_metric='mlogloss')

In [63]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)
cv_score

array([0.83311758, 0.8711081 , 0.93164808, 0.86603114, 0.78030861,
       0.72155372, 0.91036556, 0.87080739, 0.91766872, 0.92646314])

In [64]:
# 결과수집
result['XGBoost'] = cv_score.mean()

# 5.결과 확인

- 앞의 과정에서 수집한 성능 정보를 확인합니다.

In [65]:
# 평가 결과 확인
print('=' * 40)
for m_name, score in result.items():
    print(m_name, score.round(3))
print('=' * 40)

Linear Regression 0.636
KNN 0.636
Decision Tree 0.685
SVM 0.547
Random Forest 0.859
XGBoost 0.863


# 6.모델 최적화

- XGBoost 모델을 GridSearchCV를 사용해 최적화합니다.
- 파라미터 범위
    - max_depth: range(3, 11)
    - n_estimators: range(50, 111, 10)
- GridSearchCV 옵션
    - cv=5

In [79]:
# 1단계: 불러오기
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error

In [72]:
# 2단계: 선언하기
model_xgb = XGBRegressor(eval_metric='mlogloss')

# 파라미터 선언
  # n_neighbors: range(5, 51)
param = {'max_depth': range(3, 11),  'n_estimators' : range(50, 111, 10)}

model = GridSearchCV(model_xgb, param, cv=10, scoring='r2')

In [73]:
# 3단계: 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False,
                                    eval_metric='mlogloss', gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                      

In [87]:
# 결과확인(파라미너, 성능)
print(model.best_params_)
print(model.best_score_)


{'max_depth': 10, 'n_estimators': 80}
0.8679089855256074


In [88]:
df = pd.DataFrame(model.cv_results_)
df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.093502,0.010839,0.009474,0.001501,3,50,"{'max_depth': 3, 'n_estimators': 50}",0.845530,0.834075,0.929979,0.845072,0.740709,0.708673,0.889451,0.857130,0.913081,0.944486,0.850818,0.072883,47
1,0.094066,0.003946,0.008720,0.000601,3,60,"{'max_depth': 3, 'n_estimators': 60}",0.846745,0.820898,0.928976,0.842044,0.733459,0.713520,0.893273,0.860275,0.910291,0.944209,0.849369,0.073383,49
2,0.094829,0.004246,0.007261,0.000832,3,70,"{'max_depth': 3, 'n_estimators': 70}",0.849459,0.822373,0.930551,0.841676,0.735763,0.715264,0.893770,0.859448,0.911228,0.945137,0.850467,0.073026,48
3,0.135007,0.042438,0.007901,0.000763,3,80,"{'max_depth': 3, 'n_estimators': 80}",0.850434,0.824941,0.932229,0.846993,0.740097,0.717772,0.895776,0.858969,0.912007,0.943091,0.852231,0.071832,45
4,0.160398,0.019306,0.009484,0.001746,3,90,"{'max_depth': 3, 'n_estimators': 90}",0.851852,0.825077,0.932908,0.847035,0.738213,0.717992,0.897030,0.857679,0.911373,0.942743,0.852190,0.072125,46


In [76]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [81]:
# 5단계: 평가하기
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))


2.4090698888427333
0.8463536952242274
